# ipyparaview visualization


This notebook uses `ipyparaview` to visualize the outputs of the simulation.

The simulation saved out a time series data set to the `integrated_hydro*.demo/visdump_data.VisIt.xmf` file series. We can read this data with ParaView's `Xdmf3ReaderS` file reader.


Run the ipyparaview docker container (`ipp` from https://github.com/Kitware/ipyparaview/pull/1) with:

```bash
docker run -p 8888:8888 -v /path/to/ats-demos:/root/ipyparaview/notebooks/ats-demos ipp
```

In [ ]:
from paraview import simple as pvs
from ipyparaview.widgets import PVDisplay
from ipywidgets import interact
import ipywidgets as widgets
import os, glob

In [ ]:
result_directory = "integrated_hydro-v.demo"
data_file = "visdump_data.VisIt.xmf"

In [ ]:
data = pvs.Xdmf3ReaderS(FileName=os.path.join(result_directory, data_file))
data.UpdatePipeline()

In [ ]:
pvs.Show(data)
disp = PVDisplay(pvs.GetActiveView())
display(disp)

In [ ]:
data_display = pvs.GetDisplayProperties(data, view=pvs.GetActiveView())
data_display.SetScalarBarVisibility(pvs.GetActiveView(), True)

# set scalar coloring
pvs.ColorBy(data_display, ('CELLS', 'saturation_liquid.cell.0'))

# get color transfer function/color map for 'base_porositycell0'
tf = pvs.GetColorTransferFunction('saturation_liquid.cell.0')

# Rescale transfer function
tf.RescaleTransferFunction(0.509332835674, 1.0)

In [ ]:
timer = pvs.GetTimeKeeper()
anno = pvs.AnnotateTime()
pvs.Show(anno)

def set_time(change):
    step = change["new"]
    if step < 0:
        step = 0
    if step >= len(timer.TimestepValues):
        step = len(timer.TimestepValues) - 1
    timer.Time = timer.TimestepValues[step]
    return

play = widgets.Play(
    value=0,
    min=0,
    max=len(timer.TimestepValues),
    step=1,
    description="Time Step",
)

play.observe(set_time, "value")

slider = widgets.IntSlider(min=0, max=len(timer.TimestepValues), continuous_update=True)
widgets.jslink((play, 'value'), (slider, 'value'))

In [ ]:
cmin = widgets.FloatText(
    value=0.509332835674,
    description='Min:',
    disabled=False
)
cmax = widgets.FloatText(
    value=1.0,
    description='Max:',
    disabled=False
)

def set_clim(change):
    tf = pvs.GetColorTransferFunction(data_display.ColorArrayName[1])
    tf.RescaleTransferFunction(cmin.value, cmax.value)
    return

cmin.observe(set_clim, "value")
cmax.observe(set_clim, "value")


def reset_colors(*args):
    vmin, vmax = data.CellData[data_display.ColorArrayName[1]].GetRange()
    cmin.value = vmin
    cmax.value = vmax


reset = widgets.Button(
    description='Reset clim',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
)
reset.on_click(reset_colors)

In [ ]:
scalars = widgets.Dropdown(
    options=list(data.CellData.keys()), 
#                      range(len(data.CellData.keys())))),
    value=data_display.ColorArrayName[1],
    description='Scalars:',
)

def set_active_scalars(change):
    name = change["new"]
    pvs.ColorBy(data_display, ('CELLS', name))
    return reset_colors()
    
scalars.observe(set_active_scalars, 'value')
    

In [ ]:

# Final Layout
widgets.VBox([widgets.HBox([play, slider, scalars]), 
              widgets.HBox([cmin, cmax, reset]),
              disp])